## 🛠️ Data Scrapping Jupyter Notebbok

In [1]:
from bs4 import BeautifulSoup as bs 
import requests
from ultralytics import YOLO
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
from itertools import zip_longest
from requests.exceptions import HTTPError, ConnectionError, Timeout, RequestException


#### Function for detection the damage

It is using my custom made yolov8 model

In [44]:
# function which is returning the number of object detected 
def number_object_detected(image):
    custom_model = YOLO('D:/COMSATS/FYP/Coding Part/ModelMaking/Computer_Vision_Part/3rd Try/runs/detect/train9/weights/best.pt') # custome yolo model path
    results = custom_model(image,verbose= False)

    dic = results[0].names
    classes = results[0].boxes.cls.cpu().numpy()
    probability = results[0].boxes.conf
    
    class_count = {}
    unique_elements, counts = np.unique(classes, return_counts=True)
    for e , count in zip(unique_elements,counts):
        a = dic[e]
        class_count[a] = count
    """    for r in results:
        im_array = r.plot(pil = True)  # plot a BGR numpy array of predictions
        array = im_array[..., ::-1]  # Convert BGR to RGB PIL image
        plt.axis("off")
        plt.imshow(array)
        plt.show()
    """


    return class_count 

Function for getting the body Condtion 

In [45]:
severity_points = {
    'scratch': 1,
    'dent': 2,
    'rust': 3,
    'paint-damage': 4,
    'crack':4
}

def calculate_condition_score(detections):
    total_score = 0
    for detection, count in detections.items():
        if detection in severity_points:
            total_score += severity_points[detection] * count
    return total_score

def normalize_score(score, max_score):
    return (score / max_score) * 10

def estimate_condition(detections):
    print("Detedtion list",detections)
    max_possible_score = sum(severity_points.values())  # Assuming all types of damage detected
    score = calculate_condition_score(detections)
    normalized_score = normalize_score(score, max_possible_score)
    
    # Assign condition rating
    if normalized_score <= 10:  # If score is low, condition is Excellent
        return "Excellent"
    elif normalized_score <= 16:  # If score is moderately low, condition is Good
        return "Good"
    elif normalized_score <= 20:  # If score is moderate, condition is Fair
        return "Fair"
    elif normalized_score <= 258:  # If score is moderately high, condition is Poor
        return "Poor"
    else:  # If score is high, condition is Very Poor
        return "Very Poor"


Request Function

In [46]:
def request_function(url):
    req = requests.get(url).text
    soup = bs(req, features="html.parser")
    return soup


Image Scrapping function

In [47]:
## Function to get images
def image_scraper(url):
    s = request_function(url)
    image_tags = s.find_all('img', class_='lazy-used-car-slider')  
    image_urls = []
    for image_tag in image_tags:
        data_src = image_tag.get('data-original')
        if data_src.endswith('webp'):
                image_urls.append(data_src)
    return image_urls

Function to Download Image

In [48]:
## function to download images
def download_img(url):
    img_data = requests.get(url).content
    # Save the image to the specified folder
    img_name = os.path.join('images', os.path.basename('new_imaeg.jpg'))
    with open(img_name, 'wb') as img_file:
       img_file.write(img_data)
    detected_object_dic = number_object_detected(img_name)

    return detected_object_dic



In [49]:
def compiling_dic(detected_dictionary): # it is getting the dictionary of detected objects of one image from download_image_function
    compiled_dic = {}
    for key in detected_dictionary.keys():
            if key in compiled_dic:
                  compiled_dic[key] += detected_dictionary[key]
            else:
                  compiled_dic[key] = detected_dictionary[key]
    print("Compiled Dic",compiled_dic)
    return compiled_dic
    

Main Scrapping funtion 

In [62]:
ad_name = []
ad_id = []
price = []
body_condition = []
engine_capacity = []
body_type = []
model_Year=[]
millage = []
fuel_type = []
transmission = []
city_registered = []
body_color = []
ad_links = []
ad_Date  = []

In [59]:
list_of_famous_cars = ['corolla','swift' 'mehran','sportage','wagon-R', 'civic','elantra','sonata'] 
l2 = ['tucson','alto','cultus'] #fortuner','vitz','bolan','city','alsvin','mark-x',



In [63]:
def test_scrap(list_of_famous_cars):
    print('Loading Data ',end='')
    for car in list_of_famous_cars:
        for page in range(1,20):
            url = ' https://www.pakwheels.com/used-cars/search/-/?q='+car+'&page='+str(page)
            s = request_function(url)
            ads_link =  ['https://www.pakwheels.com'+a.get('href') for a in s.find_all('a',class_='car-name ad-detail-path')]
            
            for i in ads_link:
                ad_links.append(i)

                print('ad link',i)

                s2 = request_function(i)

                id = s2.find('li', string='Ad Ref #').find_next('li').get_text()
                ad_id.append(id)
                # now images
                local_img_urls = image_scraper(i)

                for img_url in local_img_urls:
                    data_dictionary = download_img(img_url)
                    compiled_dic = compiling_dic(data_dictionary)
                
                ad_name.append(s2.find('div',class_='well').find_next().text)

                body_condition.append(estimate_condition(compiled_dic))

                model_Year.append(s2.find('span', class_ = 'engine-icon year').find_next().text.strip()
                                        if s2.find( 'span', class_='engine-icon year') else 'N/A')

                millage.append(s2.find('span', class_='engine-icon millage').find_next().text.strip())

                fuel_type.append(s2.find('span', class_='engine-icon type').find_next().text.strip())

                transmission.append(s2.find('span', class_='engine-icon transmission').find_next().text.strip())


                city_registered.append(s2.find('li', string='Registered In').find_next('li').get_text(strip=True))

                body_color.append(s2.find('li', string='Color').find_next('li').get_text(strip=True))

                engine_capacity.append(s2.find('li', string='Engine Capacity').find_next('li').get_text(strip=True)
                                            if s2.find('li',string='Engine Capacity') else s2.find(
                            'li', string='Battery Capacity').find_next('li').get_text)

                body_type.append(s2.find('li', string='Body Type').find_next('li').find('a').get_text()
                                        if s2.find('li', string='Body Type').find_next('li').find('a') else 'N/A')

                ad_Date.append(s2.find('li',string = 'Last Updated:').find_next('li').get_text()
                                              if s2.find('li', string='Last Updated:').find_next('li') else 'N/A')


                price.append(s2.find('strong',class_ = 'generic-green').text)
        
        print('Converting to DataFrame')
        df = pd.DataFrame({'AD_Name':ad_name,'AD_id':ad_id,'Engine Capacity':engine_capacity,'Category':body_type,'Body Consition':body_condition,
                       "model Year":model_Year,'Milage':millage,'Fuel Type':fuel_type,'Transmission':transmission,'Color':body_color,
                       'City Registered':city_registered,'Price':price,'Ad_link':ad_links,'ad_Date':ad_Date})

        file_path = os.path.join('dataset', car+"_FYP DataSet1-to10.csv")
        df.to_csv(file_path)

        print('Saving to CSV with Name --- > FYP DataSet',df.to_csv(car+'FYP DataSet1-to10.csv'))

        

In [64]:
test_scrap(l2)



Loading Data ad link https://www.pakwheels.com/used-cars/hyundai-tucson-2023-for-sale-in-karachi-8348887
Compiled Dic {}
Compiled Dic {}
Compiled Dic {'rust': 1, 'scratch': 2}
Compiled Dic {'rust': 1, 'scratch': 2}
Compiled Dic {'paint-damage': 1}
Compiled Dic {}
Compiled Dic {'scratch': 1}
Compiled Dic {'scratch': 1}
Compiled Dic {'scratch': 1}
Compiled Dic {}
Compiled Dic {'scratch': 2}
Compiled Dic {}
Compiled Dic {'rust': 1, 'scratch': 1}
Compiled Dic {'scratch': 1}
Compiled Dic {'scratch': 2}
Compiled Dic {}
Detedtion list {}
ad link https://www.pakwheels.com/used-cars/hyundai-tucson-2020-for-sale-in-lahore-8405834
Compiled Dic {'scratch': 2}
Compiled Dic {}
Compiled Dic {'rust': 1}
Compiled Dic {'rust': 1, 'scratch': 1}
Compiled Dic {'scratch': 1}
Compiled Dic {}
Compiled Dic {}
Compiled Dic {'scratch': 1}
Compiled Dic {}
Compiled Dic {}
Detedtion list {}
ad link https://www.pakwheels.com/used-cars/changan-oshan-x7-2023-for-sale-in-lahore-7890134
Compiled Dic {'dent': 1}
Compiled

AttributeError: 'NoneType' object has no attribute 'find_next'

def converting_to_csv():
   // test_scrap(list_of_famous_cars)
    print('Converting to DataFrame')
    df = pd.DataFrame({'AD_Name':ad_name,'AD_id':ad_id,'Engine Capacity':engine_capacity,'Category':body_type,'Body Consition':body_condition,
                       "model Year":model_Year,'Milage':millage,'Fuel Type':fuel_type,'Transmission':transmission,'Color':body_color,
                       'City Registered':city_registered,'Price':price})

    print('Saving to CSV with Name --- > FYP DataSet',df.to_csv('FYP DataSet1-to10.csv'))
    print(df)
    

### I have scrapped multiple files now merging each file into one csv file

In [ ]:
# reading all the files

cwd = os.path.abspath('dataset')
file_list = os.listdir(cwd)
file_list

In [ ]:
df_con = pd.concat([pd.read_csv('dataset/'+f) for f in file_list], ignore_index= True)
df_con

In [ ]:
df_con.to_csv("Datset.csv")

In [ ]:
for f in file_list:
    print(pd.read_csv('dataset/'+f).shape)

In [ ]:
import pandas as pd
import random

# Load the CSV file
df = pd.read_csv('your_file.csv')

# Add a new column named "data" with NaN values
df['data'] = pd.NA

# Randomly assign values "2023" and "2022" to the first 50 rows
for index in range(50):
    df.at[index, 'data'] = random.choice(["2023", "2022"])

# Assign "2024" to the rest of the rows
df.loc[50:, 'data'] = "2024"

# Save the modified DataFrame to a new CSV file
df.to_csv('modified_file.csv', index=False)


### Getting the dates again

In [1]:
alsvin = pd.read_csv("alsvinFYP DataSet1-to10.csv")

C:\Users\M_ros\AppData\Local\Temp\ipykernel_15744\4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
def scrape_data_from_link(link):
    dates = []
    try:
        for i in link:
          response = requests.get(i).text
          s2 = bs(response)
          # Extract the data you need from the webpage
          dates.append(s2.find('li',string = 'Last Updated:').find_next('li').get_text()
                                              if s2.find('li', string='Last Updated:').find_next('li') else 'N/A')

        return dates
    
    except Exception as e:
        print(f"Error scraping data from {link}: {e}")
        return None


In [8]:
files = ["alsvinFYP DataSet1-to10.csv","cityFYP DataSet1-to10.csv"]
for i in files:
    d = pd.read_csv(i)
    dates = scrape_data_from_link(d['Ad_link'])
    d['ad_Date '] = dates
    d.to_csv('withdates'+i)
